----

<a
  target="_blank" href="https://colab.research.google.com/drive/10w9E1whCPMZZposk2vru9cYNEA3CavuG">
  <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
  Ouvrir dans Google Colab
</a>

----

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import math
import numpy as np
import matplotlib.pyplot as plt

dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

class_names = metadata.features['label'].names
print("Labels des vêtements : {}".format(class_names))

num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples

print("Nombre d'exemples pour l'entrainement : {}".format(num_train_examples))
print("Nombre d'exemples pour le test final : {}".format(num_test_examples))

def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

train_dataset =  train_dataset.map(normalize)
test_dataset  =  test_dataset.map(normalize)
train_dataset =  train_dataset.cache()
test_dataset  =  test_dataset.cache()

for image, label in test_dataset.take(1):
  break
image = image.numpy().reshape((28,28))
plt.figure()
plt.imshow(image, cmap=plt.cm.binary)
plt.grid(False)
plt.show()

plt.figure(figsize=(10,10))

for i, (image, label) in enumerate(test_dataset.take(25)):
    image = image.numpy().reshape((28,28))
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image, cmap=plt.cm.binary)
    plt.xlabel(class_names[label])

plt.show()

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

lots_img = 32
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(lots_img)
test_dataset = test_dataset.cache().batch(lots_img)
model.fit(train_dataset, epochs=3, steps_per_epoch=math.ceil(num_train_examples/lots_img))

test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/50))
print('Précision du modèle :', int(round(test_accuracy, 2)*100),'%')

for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

print(predictions[20])
print('Probabilité la plus haute : étiquette de vêtement numéro', np.argmax(predictions[20]))
print('Réponse correcte : étiquette de vêtement numéro', test_labels[20])
print('Vêtement détecté :', class_names[0])

def plot_image(i, predictions_array, true_labels, images):
  predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(img[...,0], cmap=plt.cm.binary)
  predicted_label = np.argmax(predictions_array)

  if predicted_label == true_label:
    color = 'green'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="lightgreen")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('green')

i = 5
plt.figure(figsize=(15,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

nbr_lignes = 3
nbr_col = 3
nbr_imgs = nbr_lignes * nbr_col
plt.figure(figsize=(16, 8))
for i in range(nbr_imgs):
  plt.subplot(nbr_lignes, 2*nbr_col, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(nbr_lignes, 2*nbr_col, 2*i+2)
  plot_value_array(i, predictions, test_labels)